In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
!pip install sklearn-crfsuite

In [ ]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

## Let's use CoNLL 2002 data to build a NER system

CoNLL2002 corpus is available in NLTK. We use Spanish data.

In [ ]:
import nltk
nltk.download('conll2002')
nltk.corpus.conll2002.fileids()

In [ ]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

In [ ]:
print('wielkosć zbioru treningowego:', len(train_sents))
print('wielkosć zbioru testowego:' , len(test_sents))
train_sents[10]
# opis znaczeń tagów: https://freeling-user-manual.readthedocs.io/es/latest/tagsets/tagset-en/

In [ ]:
nltk.help #używany w nltk tagset angielski

In [ ]:
#proszę wylistować wszystkie występujące w zbiorze treningowym tagi (2 kolumna) i etykiety (3 kolumna)
labl={}
tagl={}
for s in train_sents:
    for w in s:
        try:
            labl[w[2]]+=1
        except KeyError:
            labl[w[2]]=1
        try:
            tagl[w[1]]+=1
        except KeyError:
            tagl[w[1]]=1
#labl=sorted(labl)
#tagl=sorted(tagl)

In [ ]:
print(labl)
print(tagl)
print(sorted(labl))
print(sorted(tagl))

## Features

Next, define some features. In this example we use word identity, word suffix, word shape and word POS tag; also, some information from nearby words is used. 

This makes a simple baseline, but you certainly can add and remove some features to get (much?) better results - experiment with it.

sklearn-crfsuite (and python-crfsuite) supports several feature formats; here we use feature dicts.

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
      
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
        })
    else:
        features['BOS'] = True
  
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
        })
    else:
        features['EOS'] = True
    
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

This is what word2features extracts:

In [ ]:
print(train_sents[0])
sent2features(train_sents[0])[1]

Extract features from the data:

In [ ]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

## Training

To see all possible CRF parameters check its docstring. Here we are useing L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

In [ ]:
 #help(sklearn_crfsuite.CRF)

In [ ]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

## Evaluation

There is much more O entities in data set, but we're more interested in other entities. To account for this we'll use averaged F1 score computed for all labels except for O. ``sklearn-crfsuite.metrics`` package provides some useful metrics for sequence classification task, including this one.

In [ ]:
labels = list(crf.classes_)
labels.remove('O')
labels

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

Inspect per-class results in more detail:

In [ ]:
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

## Hyperparameter Optimization

To improve quality try to select regularization parameters using randomized search and 3-fold cross-validation.

I takes quite a lot of CPU time and RAM (we're fitting a model ``50 * 3 = 150`` times), so grab a tea and be patient, or reduce n_iter in RandomizedSearchCV, or fit model only on a subset of training data.

In [ ]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=20, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=10,   #zmienione 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Best result:

In [ ]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

### Check parameter space

A chart which shows which ``c1`` and ``c2`` values have RandomizedSearchCV checked. Red color means better results, blue means worse.

In [ ]:
_x = [s['c1'] for s in rs.cv_results_['params']]
_y = [s['c2'] for s in rs.cv_results_['params']]
_c = [s for s in rs.cv_results_['mean_test_score']]

fig = plt.figure()
fig.set_size_inches(12, 12)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('C1')
ax.set_ylabel('C2')
ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
    min(_c), max(_c)
))

ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

## Check best estimator on our test data

As you can see, quality is improved.

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

## Let's check what classifier learned

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

We can see that, for example, it is very likely that the beginning of an organization name (B-ORG) will be followed by a token inside organization name (I-ORG), but transitions to I-ORG from tokens with other labels are penalized.

Check the state features:

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])



Some observations: (from previous runs, more iterations)

   * the model remembered names of some entities - maybe it is overfit, or maybe our features are not adequate, or maybe remembering is indeed helpful;
   * "calle" is a street in Spanish; model learns that if a previous word was "calle" then the token is likely a part of location;
   *  UPPERCASED or TitleCased words are likely entities of some kind;
   * proper nouns (NP is a proper noun in the Spanish tagset) are often entities.

What to do next

    * Load 'testa' Spanish data.
    * Use it to develop better features and to find best model parameters.
    * Apply the model to 'testb' data again.





In [ ]:
#krosswalidacja
from sklearn.model_selection import KFold
import numpy

A = numpy.array([[1, 2], [3, 4], [1, 2], [3, 4], [5,6],[7,8]])
b = numpy.array([1, 2, 3, 4,5,6])
kf = KFold(n_splits=3)
kf.get_n_splits(A)
print(kf)
#KFold(n_splits=2, random_state=None, shuffle=False)
for train_index, test_index in kf.split(A):
     print("TRAIN:", train_index, "TEST:", test_index)
     b_train, b_test = b[train_index], b[test_index]

In [ ]:
kf = KFold(n_splits=3)
kf.get_n_splits(X_train)
y_pred_all=[]
for train_index, test_index in kf.split(X_train):
    X_tr, X_tst = numpy.array(X_train)[train_index], numpy.array(X_train)[test_index]
    y_tr, y_tst = numpy.array(y_train)[train_index], numpy.array(y_train)[test_index]
    #print(train_index[1])
    #print(X_train[test_index[0]])
    crf.fit(X_tr, y_tr)
    y_pred = crf.predict(X_tst)
    #print("fold")
    #print(metrics.flat_classification_report(y_tst, y_pred, labels=sorted_labels, digits=3))
    y_pred_all.extend(y_pred)
print("all results")
print(metrics.flat_classification_report(y_train, y_pred_all, labels=sorted_labels, digits=3))
  
  

Budowa końcowego modelu na całych danych treningowych i testowanie na danych testowych

In [ ]:
crf.fit(X_train, y_train)
y_pred = crf.predict(X_test)
print("wynik na danych testowych")
print(metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))

#Zadanie: 


1. Znajdz najlepsze rozwiązanie (skup sie na wyborze cech, a nie optymalizacji parametrów) dla zbioróe traina i testa
